In [1]:
import numpy as np
import pandas as pd
import music21
import re
from tqdm import tqdm

# set a random seed
np.random.seed(42)

In [2]:
data_path = "data/"
collections = ["ONeillsJigs", "folkrnnv2bs_100001jigs"]

In [3]:
dataframes = {}

for collection_name in collections:
    FILENAME = data_path + "collection_" + collection_name
    print("Processing", FILENAME)
    with open(FILENAME, encoding="utf-8") as f:
        data = f.read()
    files = data.split("\n\n")
    dictionary = {"title": [], "time_signature": [], "key": [], "abcdata": [], "abc_raw": []}
    for f in files:
        regexp = r"^(T:)?(?P<title>.*)?(\nM:)?(?P<time_signature>.*)?(\nK:)?(?P<key>.*)?(\n)?(?P<abcdata>.*)?$"
        m = re.match(regexp, f, re.M)
        d = m.groupdict()
        [dictionary[k].append(v) for k, v in d.items()]
        dictionary["abc_raw"].append(f)

    df = pd.DataFrame.from_dict(dictionary)
    
    dataframes[collection_name] = df

Processing data/collection_ONeillsJigs
Processing data/collection_folkrnnv2bs_100001jigs


In [4]:
for collection_name, df in dataframes.items():
    empty_df = pd.DataFrame(columns=["title", "time_signature", "key", "abcdata"])

    if len(df) > 365:
        while len(empty_df) < 365:
            # randomly sample a song
            song = df.sample(1)
            if song.index in [43341, 78322, 25461]:
                # this song is not valid, should be fixed in the dataset
                continue
            abcstr = (
                "X:1\nM:"
                + song.time_signature.values[0]
                + "\nK:"
                + song.key.values[0]
                + "\n"
                + "".join(song.abcdata.values[0].split())
            )
            s1 = music21.converter.parseData(abcstr)
            if ":|" in abcstr:
                try:
                    # count how many times the repeat occurs
                    s1 = s1.expandRepeats()
                except music21.repeat.ExpanderException as e:
                    # the tune is not valid, should be fixed in the dataset
                    print("Error in expanding repeats for song", song.title.values[0])
                    continue
            # append the song
            empty_df = pd.concat([empty_df, song], ignore_index=True)
            df.drop(song.index, inplace=True)

        df = empty_df
        dataframes[collection_name] = df

Error in expanding repeats for song 51352
Error in expanding repeats for song 41267


In [5]:
# %% compute features
# determining the sampling rate Fs (samples per quaver): 
# 1. the smallest time interval in the collection is triplet semiquavers, 
#    which means Fs should be a multiple of 3
# 2. I also want a semiquaver to have a whole number of samples, so Fs 
#    should be a multiple of 2
# 3. These mean Fs should be a multiple of 3*2 = 6. Let's make Fs=6
# This makes an 8-measure part become a time series of length 6*6*8 = 288

Fs = 6.0 # samples per quaver

for collection_name, df in dataframes.items():
    TimePitchTune = []
    TimePitchBars = []
    timespace = []
    pitchspace = []

    print("Processing", collection_name)
    for ii in tqdm(df.index):
        # create ABC string
        abcstr = 'X:1\nM:'+df.time_signature[ii]+'\nK:'+df.key[ii]+'\n'+"".join(df.abcdata[ii].split())
        # parse ABC string to music21 stream
        s1 = music21.converter.parseData(abcstr)
        # make repetitions explicit
        if ":|" in abcstr:
            s1 = s1.expandRepeats()
        # extract pitches and durations 
        pitches = []; durrep = [0]; beats = []
        prevpitch = 0
        for event in s1.flatten().notesAndRests:
            if type(event) == music21.note.Note:
                pitches.append(event.pitch.ps)
                prevpitch = event.pitch
            else:
                pitches.append(prevpitch.ps)
                
            # take care of durations expressed as a fraction
            if type(event.duration.quarterLength) == music21.common.numberTools.Fraction:
                frac = event.duration.quarterLength
                durrep.append(2*frac.numerator/frac.denominator)
            else:
                durrep.append(2*event.duration.quarterLength)
                
            if type(event.beat) == music21.common.numberTools.Fraction:
                frac = event.beat
                beats.append(3*frac.numerator/frac.denominator-2)
            else:
                beats.append(3*event.beat-2)

        # account for anacrusis
        if beats[0] != 1:
            num2trim = int(6-beats[0]+1)
            # append the elements
            for p in pitches[1:num2trim+1]:
                pitches.append(p)
            for d in durrep[1:num2trim+1]:
                durrep.append(d)
            # remove them
            pitches = [pitches[0]] + pitches[num2trim+1:]
            durrep = [durrep[0]] + durrep[num2trim+1:] 

        # compute melody profile
        nppsrep = np.array(pitches)
        ts = np.cumsum(np.array(durrep)) # timespace representation

        pitchspace.append(nppsrep)
        timespace.append(ts)

        X = []; y = []
        
        for jj in range(len(ts)-1):
            X.append(ts[jj]); X.append(ts[jj+1])
            y.append(nppsrep[jj]); y.append(nppsrep[jj])
            
        # interpolate
        X = np.asarray(X); y = np.asarray(y)
        from sklearn.neighbors import KNeighborsRegressor
        interpolator = KNeighborsRegressor(1)
        interpolator.fit(X.reshape(-1, 1),y)
        X = np.arange(0,np.max(ts),1.0/Fs)
        PitchRep = interpolator.predict(X.reshape(-1, 1))

        TimePitchTune.append(PitchRep)

        # break it up into measures (6 * Fs samples)
        try:
            # Attempt to reshape
            TimePitchBars.append(PitchRep.reshape(-1, int(6 * Fs)))
        except ValueError:
            print(f"Error in reshaping song {df.title[ii]}, PitchRep length: {len(PitchRep)}. Padding the end with zeros.")
            # Optionally, pad or trim
            PitchRep = np.pad(PitchRep, (0, int(6 * Fs) - len(PitchRep) % int(6 * Fs)), mode='constant')
            TimePitchBars.append(PitchRep.reshape(-1, int(6 * Fs)))

    df['timespace'] = timespace
    df['pitchspace'] = pitchspace
    df['TimePitchTune'] = TimePitchTune
    df['TimePitchBars'] = TimePitchBars

    df.to_pickle(data_path + collection_name +'_parsed.pkl')

Processing ONeillsJigs


  5%|▌         | 19/365 [00:02<00:30, 11.48it/s]

Error in reshaping song Saddle the pony, PitchRep length: 1158. Padding the end with zeros.


  7%|▋         | 25/365 [00:02<00:21, 15.69it/s]

Error in reshaping song The night cap, PitchRep length: 1158. Padding the end with zeros.


...

 92%|█████████▏| 337/365 [00:21<00:01, 16.39it/s]

Error in reshaping song 71397, PitchRep length: 1146. Padding the end with zeros.
Error in reshaping song 6227, PitchRep length: 1716. Padding the end with zeros.


 98%|█████████▊| 356/365 [00:22<00:00, 16.13it/s]

Error in reshaping song 87239, PitchRep length: 1698. Padding the end with zeros.
Error in reshaping song 25144, PitchRep length: 1740. Padding the end with zeros.


100%|██████████| 365/365 [00:22<00:00, 15.97it/s]
